In [18]:
from bokeh.plotting import output_notebook, show
from bokeh.models import LinearColorMapper, ColorBar
from generic_create_bokeh_fig import create_generic_fig
import numpy as np
import xarray as xr

# This is a simple notebook to demonstrate the data analysis workflow with xrDataArrays and Bokeh for plotting.
# A small dataset has been randomly generated and plotted to show functionality.

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
from bokeh.palettes import Viridis256

In [15]:
# create a simple 4-dimensional dataset of random numbers between 0-1
# give the dimensions names, similar to pandas dataframes, so they can be recalled by name or number
# also supply coordinates, specific indices within a dimension, for the measured quantity dimension with names (voltage and current)
# include attributes, or metadata, in the xrDataArray

data = xr.DataArray(
    np.random.rand(5, 5, 2, 4),
    dims=("row", "col", "measured_quantity", "cycle_num"),
    coords={
        "measured_quantity": [
            "voltage",
            "current",
        ]
    },
    attrs=dict(
        measurement_title="test",
        measurement_time="2 s",
    ),
)

In [16]:
# instantiate a figure using the generic figure function created for Bokeh
# create a scatter plot by retrieving all the voltage and current values contained within the xrDataArray
# and plotting by flattening the array to 1-dimensional

fig1 = create_generic_fig("Voltage", "Current", frame_height=350, frame_width=350)

fig1.scatter(
        data.sel(measured_quantity="voltage").values.flatten(),
        data.sel(measured_quantity="current").values.flatten(), size=12, marker='square'
    )

show(fig1)

In [17]:
# first create a grid and retrieve the row/column numbers as a flattened array for rows (r) and columns (c)
# get a slice of the 4-dimensional xrDataArray that shows the 5x5 array for the current on the 3rd cycle
# map the randomized values between 0-1 onto the Viridis256 color palette provided by Bokeh
# and plot the corresponding color as a square on the 5x5 array
# include a colorbar to the right of the figure to indicate what values the colors represent

r, c = np.mgrid[1:6, 1:6]
r = r.flatten()
c = c.flatten()

plot_data = dict(rows=r, columns=c, val=data.sel(measured_quantity="current", cycle_num=3).values.flatten())

fig2 = create_generic_fig("Column", "Row", frame_height=350, frame_width=350, x_range=(0.5, 5.5), y_range=(0.5, 5.5))

mapper = LinearColorMapper(palette=Viridis256, low=0, high=1)

fig2.rect("columns", "rows", source=plot_data, width=1, height=1, fill_color={"field": "val", "transform": mapper})

color_bar = ColorBar(
    color_mapper=mapper,
    location=(10, 0),
    bar_line_color="black",
    bar_line_width=3,
    width=int(fig2.frame_width * (12 / 225)),
    major_label_text_font_size="{}px".format(fig2.frame_width * (12 / 225)),
    major_label_text_color="black",
    label_standoff=int(fig2.frame_width * (6 / 225)),
    major_tick_line_color="black",
    major_tick_line_width=3,
    major_tick_in=10,
)

fig2.add_layout(color_bar, "right")

show(fig2)